# Import necessary libraries

In [3]:
import pandas as pd

# Load The MTN Upsell dataset

In [66]:
df = pd.read_excel('../data/raw/mtn_upsell_uncleaned.xlsx')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9377 entries, 0 to 9376
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dates                9377 non-null   int64  
 1   Phone Number         9377 non-null   int64  
 2   total_reloads        9377 non-null   int64  
 3   total_reload_amount  9309 non-null   float64
 4   imei                 9377 non-null   int64  
 5   brand_name           9377 non-null   object 
 6   model_name           9377 non-null   object 
 7   device_category      9377 non-null   object 
 8   data_kb              9377 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 659.4+ KB


# Data Cleaning

As you see we have 60+ nan rows in 'total_reload_amount' column. Common causes are

1. System recording error – Reload was counted, but the amount wasn’t saved.

2. Free bonus/airdrop – A “reload” without money involved (rare).

3. Incomplete data – Some recharge transactions didn’t sync fully.

So i think it's an incomplete data problem, i decide to impute those nan values by the mean of people who reloaded 1 time

In [67]:
mean_total_reloads_amount = df[(df['total_reloads'] == 1) & (df['total_reload_amount'].notna())]['total_reload_amount'].mean()
float(mean_total_reloads_amount)

341.4826373038516

In [68]:
df['total_reload_amount'] = df['total_reload_amount'].fillna(mean_total_reloads_amount)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9377 entries, 0 to 9376
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dates                9377 non-null   int64  
 1   Phone Number         9377 non-null   int64  
 2   total_reloads        9377 non-null   int64  
 3   total_reload_amount  9377 non-null   float64
 4   imei                 9377 non-null   int64  
 5   brand_name           9377 non-null   object 
 6   model_name           9377 non-null   object 
 7   device_category      9377 non-null   object 
 8   data_kb              9377 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 659.4+ KB
